# Identificador de hongos

## Entendimiento de los datos

El dataset que se utiliza es *Mushrooms*, por Derek Kuno-Williams (2022), recuperado de [Kaggle](https://doi.org/10.34740/KAGGLE/DSV/3690937) en febrero del 2023. Este dataset contiene 8781 imágenes de hongos organizadas en subdirectorios que indican su clasificación y taxonomía, siendo las clasificaciones **"mortífero"**, **"venenoso"**, **"condicionalmente comestible"**, y **"comestible"**, y su taxonomía representada mediante **género** y **especie**. Cabe remarcar que las clasificaciones pueden ser incorrectas, por lo que no se debe utilizar para determinar si un hongo dado puede ser ingerido de forma segura.

## Preparación de los datos

La base de datos se crea con las siguientes tablas de información general:

- **mushrooms**: Cada registro contiene la imágen original, una versión de la imágen original con la paleta adaptativa, taxonomía (FK en *taxonomy*), clasificación.
- **taxonomies**: Cada registro contiene género y especie.
- **links_en**: Cada registro contiene un enlace a Wikipedia (EN) para cada registro en *taxonomy*.
- **links_es**: Cada registro contiene un enlace a Wikipedia (ES) para cada registro en *taxonomy*.

Para realizar la búsqueda se crean las siguientes tablas de vectores característicos:

- **histograms**: Cada registro contiene los histograms que se obtienen de hacer binning (64×3) a las frecuencias relativas de las componentes para cada canal de la imágen original. Se tiene una columna por modo de color, y los valores son los vectores serializados.
- **palette**: Cada registro representa uno de los colores de la paleta adaptativa a la que se convierten las imágenes.
- **palette_hist**: Cada registro contiene el histograma que se obtiene de las frecuencias relativas de cada color de la paleta presente en la versión de la imágen que utiliza la paleta adaptativa.

In [22]:
import sqlite3

queries = [
    "CREATE TABLE taxonomies (id INTEGER PRIMARY KEY AUTOINCREMENT, genus text NOT NULL, species text, UNIQUE(genus, species)) STRICT",
    "CREATE TABLE mushrooms (id INTEGER PRIMARY KEY AUTOINCREMENT, picture blob NOT NULL, adapted blob, taxonomy_id INTEGER NOT NULL, classification TEXT NOT NULL, CONSTRAINT fk_mushrooms_taxonomies FOREIGN KEY (taxonomy_id) REFERENCES taxonomies(id))",
    "CREATE TABLE links_en (id INTEGER PRIMARY KEY AUTOINCREMENT, taxonomy_id INTEGER NOT NULL, link TEXT, CONSTRAINT fk_links_en_taxonomies FOREIGN KEY (taxonomy_id) REFERENCES taxonomies(id))",
    "CREATE TABLE links_es (id INTEGER PRIMARY KEY AUTOINCREMENT, taxonomy_id INTEGER NOT NULL, link TEXT, CONSTRAINT fk_links_es_taxonomies FOREIGN KEY (taxonomy_id) REFERENCES taxonomies(id))",
]

con = sqlite3.connect("mushrooms.sqlite")
try:
    for query in queries:
        con.execute(query)
except sqlite3.DatabaseError as ex:
    print("Exception: ", ex)
con.close()

In [23]:
import os
from PIL import Image
from ipywidgets import IntProgress
from IPython.display import display

DATASETS_PATH = r"mushroom_dataset"

def mushroom_file_generator():
    for classification in os.listdir(DATASETS_PATH):
        for taxonomy in os.listdir(DATASETS_PATH + "\\" + classification):
            for file in os.listdir(
                DATASETS_PATH + "\\" + classification + "\\" + taxonomy
            ):
                taxonomy_split = taxonomy.split("_")
                yield {
                    "classification": classification,
                    "genus": taxonomy_split[0],
                    "species": taxonomy_split[1] if len(taxonomy_split)>1 else "",
                    "file_path": DATASETS_PATH + "\\" + classification + "\\" + taxonomy + "\\" + file,
                }
                
mg = mushroom_file_generator()
shrooms = list(mg)

f = IntProgress(min=0, max=len(shrooms))
display(f)

for i, shroom in enumerate(shrooms):
    
    if i == 0 or shroom["genus"] != genus or shroom["species"] != species:
        genus = shroom["genus"]
        species = shroom["species"]
        con = sqlite3.connect("mushrooms.sqlite")
        cur = con.cursor()
        try:
            with con:
                cur.execute("INSERT INTO taxonomies (genus, species) VALUES (:genus, :species)", {"genus": genus, "species": species})
                con.commit()
        except sqlite3.DatabaseError as ex:
            print("tax Exception: ", ex, i, shroom)
        con.close()
    classification = shroom["classification"]
    with open(shroom["file_path"], "rb") as input_file:
        picture = input_file.read()
        con = sqlite3.connect("mushrooms.sqlite")
        cur = con.cursor()
        try:
            with con:
                cur.execute("SELECT id FROM taxonomies WHERE genus = ? AND species = ?", (shroom["genus"], shroom["species"]))
                tax = cur.fetchone()[0]
                con.execute("INSERT INTO mushrooms (picture, taxonomy_id, classification) VALUES (?, ?, ?)", (sqlite3.Binary(picture), tax, classification))
        except sqlite3.DatabaseError as ex:
            print("mush Exception: ", ex, i, shroom, tax)
        except TypeError as ex:
            print("mush Exception: ", ex, i, tax)
        con.close()

IntProgress(value=0, max=8781)

tax Exception:  UNIQUE constraint failed: taxonomies.genus, taxonomies.species 5338 {'classification': 'poisonous', 'genus': 'Amanita', 'species': 'smithiana', 'file_path': 'mushroom_dataset\\poisonous\\Amanita_smithiana\\Amanita_smithiana0.png'}
tax Exception:  UNIQUE constraint failed: taxonomies.genus, taxonomies.species 5530 {'classification': 'poisonous', 'genus': 'Armillaria', 'species': 'mellea', 'file_path': 'mushroom_dataset\\poisonous\\Armillaria_mellea\\Armillaria_mellea0.png'}
tax Exception:  UNIQUE constraint failed: taxonomies.genus, taxonomies.species 5734 {'classification': 'poisonous', 'genus': 'Clitocybe', 'species': 'dealbata', 'file_path': 'mushroom_dataset\\poisonous\\Clitocybe_dealbata\\Clitocybe_dealbata0.png'}
tax Exception:  UNIQUE constraint failed: taxonomies.genus, taxonomies.species 5930 {'classification': 'poisonous', 'genus': 'Coprinopsis', 'species': 'atramentaria', 'file_path': 'mushroom_dataset\\poisonous\\Coprinopsis_atramentaria\\Coprinopsis_atrament